### Preprocess Climate Data

In [8]:
import pandas as pd

file_path = 'data/climate_data.csv'

climate_df = pd.read_csv(file_path)
climate_df['date'] = pd.to_datetime(climate_df['date'])
climate_df.index = climate_df['date']

In [2]:
from sklearn.preprocessing import MinMaxScaler

numeric_cols = climate_df.select_dtypes(include=['float64', 'int64']).columns.to_list()
numeric_cols.remove('lat')
numeric_cols.remove('lon')
non_numeric_cols = climate_df.select_dtypes(exclude=['float64', 'int64']).columns.to_list() + ['lat', 'lon']

climate_df[numeric_cols] = climate_df[numeric_cols].astype(float)
climate_df[non_numeric_cols] = climate_df[non_numeric_cols]
scaler = MinMaxScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(climate_df[numeric_cols]), columns=numeric_cols, index=climate_df.index)

climate_data = pd.concat([scaled_df, climate_df[non_numeric_cols]], axis=1).reset_index(drop= True)

### Preprocess Energy Data

In [ ]:
file_path = 'data/long_data_.csv'

energy_df = pd.read_csv(file_path)
energy_df['Dates'] = pd.to_datetime(energy_df['Dates'], format='%d/%m/%Y %H:%M:%S')
energy_df.index = energy_df['Dates']

redundant_cols = ["Regions"]
energy_df = energy_df.drop(columns=redundant_cols)

In [4]:
numeric_cols = energy_df.select_dtypes(include=['float64', 'int64']).columns.to_list()
numeric_cols.remove('latitude')
numeric_cols.remove('longitude')
non_numeric_cols = energy_df.select_dtypes(exclude=['float64', 'int64']).columns.to_list() + ['latitude', 'longitude']
energy_df[numeric_cols] = energy_df[numeric_cols].astype(float)

scaler = MinMaxScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(energy_df[numeric_cols]), columns=numeric_cols, index=energy_df.index)
energy_data = pd.concat([scaled_df, energy_df[non_numeric_cols]], axis=1).reset_index(drop=True)

### State-Lat-Lon Map

In [5]:
import numpy as np 

lat_range = np.array([10, 14, 18, 22, 26, 30, 34])
lon_range = np.array([72, 76, 80, 84, 88])

energy_data["latitude"] = energy_data["latitude"].apply(lambda x: lat_range[np.argmin(np.abs(lat_range - x))])
energy_data["longitude"] = energy_data["longitude"].apply(lambda x: lon_range[np.argmin(np.abs(lon_range - x))])

## PreTrain on Climate Data

### Time-Series Sequences

In [6]:
import numpy as np

def lagged_climate_input(data, seq_length=28):
    X = []
    for _, group in data.groupby(["lat", "lon"]):
        group = group.sort_values("date")
        features = group.drop(columns=["date", "lat", "lon"])
        for row in range(len(features) - seq_length):
            X.append(features.iloc[row:row+seq_length].values)
    return np.array(X)

climate_X = lagged_climate_input(climate_data, 14)

### Model

In [ ]:
import torch
import torch.nn as nn

class ClimatePreTrainer(nn.Module):
    def __init__(self, input_size, hidden_size=64, latent_size=32, num_layers=1, dropout=0.3):
        super(ClimatePreTrainer, self).__init__()
        
        self.encoder = nn.LSTM(input_size, hidden_size, num_layers=num_layers, 
                               batch_first=True, bidirectional=True, dropout=dropout)
        
        self.to_latent = nn.Linear(hidden_size * 2, latent_size)
        self.from_latent = nn.Linear(latent_size, hidden_size * 2)
        
        self.decoder = nn.LSTM(hidden_size * 2, hidden_size, num_layers=num_layers, 
                               batch_first=True, bidirectional=True, dropout=dropout)
        
        self.output_layer = nn.Linear(hidden_size * 2, input_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        encoded_x, _ = self.encoder(x)
        encoded_x = self.dropout(encoded_x)
        
        latent = self.to_latent(encoded_x)
        expanded = self.from_latent(latent)
        
        decoded_x, _ = self.decoder(expanded)
        decoded_x = self.dropout(decoded_x)
        
        reconstructed_x = self.output_layer(decoded_x)
        return reconstructed_x, latent


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torchinfo import summary

num_epochs = 50
batch_size = 32
lr = 0.001

patience = 5
best_val_loss = float('inf')
counter = 0

climate_X_tensor = torch.tensor(climate_X, dtype=torch.float32)

train_X, val_X = train_test_split(climate_X_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(train_X)
val_dataset = TensorDataset(val_X)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
climate_pretrained_model = ClimatePreTrainer(input_size=climate_X.shape[2], num_layers=2).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(climate_pretrained_model.parameters(), lr=lr, weight_decay=1e-5)

for epoch in range(num_epochs):
    climate_pretrained_model.train()
    train_loss = 0
    for batch in train_loader:
        batch = batch[0].to(device)
        optimizer.zero_grad()
        
        reconstructed_x, _ = climate_pretrained_model(batch)
        loss = criterion(reconstructed_x, batch)
        
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)
    
    climate_pretrained_model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = batch[0].to(device)
            reconstructed_x, _ = climate_pretrained_model(batch)
            loss = criterion(reconstructed_x, batch)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")
    
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
        torch.save(climate_pretrained_model.state_dict(), 'climate_pretrained_model.pth')
        print(f"Validation loss improved. Saving model.")
    else:
        counter += 1
        print(f"Validation loss did not improve. Patience Counter: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered.")
            break


Model Summary:
ClimatePreTrainer(
  (encoder): LSTM(11, 64, batch_first=True, bidirectional=True)
  (decoder): LSTM(128, 64, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=128, out_features=11, bias=True)
)
Layer (type:depth-idx)                   Output Shape              Param #
ClimatePreTrainer                        [14, 11]                  --
├─LSTM: 1-1                              [14, 128]                 39,424
├─LSTM: 1-2                              [14, 128]                 99,328
├─Linear: 1-3                            [14, 11]                  1,419
Total params: 140,171
Trainable params: 140,171
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 248.66
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 0.56
Estimated Total Size (MB): 0.59
Epoch [1/1], Loss: 0.0000


#### Freeze Encoder Model

In [ ]:
for param in climate_pretrained_model.parameters():
    param.requires_grad = False

## FineTune Downstream

### Merge DataSources

In [76]:
climate_data["month"] = climate_data["date"].dt.month
climate_data["year"] = climate_data["date"].dt.year
climate_data = climate_data.drop(columns=["date"])
climate_monthly = climate_data.groupby(["lat", "lon", "year", "month"]).mean().reset_index()


In [77]:
energy_data["month"] = energy_data["Dates"].dt.month
energy_data["year"] = energy_data["Dates"].dt.year
energy_data = energy_data.drop(columns=["Dates"])

energy_data = energy_data.rename(columns={"latitude": "lat", "longitude": "lon"})

merged_df = pd.merge(
    energy_data,
    climate_monthly,
    how="left",
    left_on=["lat", "lon", "year", "month"],
    right_on=["lat", "lon", "year", "month"],
    suffixes=("", "_climate"),
)

In [78]:
from sklearn.preprocessing import LabelEncoder
categorical_cols = merged_df.select_dtypes(include=["object"]).columns
for col in categorical_cols:
    le = LabelEncoder()
    merged_df[col] = le.fit_transform(merged_df[col])


In [80]:
def lagged_merged_data(data, seq_length=28):
    X = []
    y = []
    for _, group in data.groupby(["lat", "lon"]):
        group = group.sort_values("year")
        features = group.drop(columns=["year", "month", "lat", "lon", "States", "Usage"])
        target = group["Usage"]
        for row in range(len(features) - seq_length):
            X.append(features.iloc[row:row+seq_length].values)
            y.append(target.iloc[row+seq_length])
    return np.array(X), np.array(y)

merged_X, targt_y = lagged_merged_data(merged_df, 14)

In [81]:
class EnergyPrediction(nn.Module):
    def __init__(self, encoder, input_size, hidden_size=64):
        super(EnergyPrediction, self).__init__()
        self.encoder = encoder.encoder
        self.regressor = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )

    def forward(self, x):
        encoded_x, _ = self.encoder(x)  # [batch, seq_len, hidden*2]
        x = self.regressor(encoded_x[:, -1, :])  # Use last timestep
        return x  # [batch, 1]


In [82]:
num_epochs = 1
batch_size = 32
lr = 0.001

train_dataset = TensorDataset(torch.tensor(merged_X, dtype=torch.float32), torch.tensor(targt_y, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

energy_model = EnergyPrediction(climate_pretrained_model, input_size=merged_X.shape[2]).to(device)
print("Model Summary:")
print(energy_model)
print(summary(energy_model, input_size=(1, 14, merged_X.shape[2])))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(energy_model.parameters(), lr=lr)
energy_model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        inputs, targets = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()
        outputs = energy_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Model Summary:
EnergyPrediction(
  (encoder): LSTM(11, 64, batch_first=True, bidirectional=True)
  (regressor): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)
Layer (type:depth-idx)                   Output Shape              Param #
EnergyPrediction                         [1, 1]                    --
├─LSTM: 1-1                              [1, 14, 128]              39,424
├─Sequential: 1-2                        [1, 1]                    --
│    └─Linear: 2-1                       [1, 64]                   8,256
│    └─ReLU: 2-2                         [1, 64]                   --
│    └─Linear: 2-3                       [1, 1]                    65
Total params: 47,745
Trainable params: 47,745
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.56
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.19
Estimated Total Size (MB): 0.21


/home/kvv/miniconda3/envs/nn/lib/python3.12/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/1], Loss: nan


/home/kvv/miniconda3/envs/nn/lib/python3.12/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([27])) that is different to the input size (torch.Size([27, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [83]:
for param in energy_model.encoder.parameters():
    param.requires_grad = True

for epoch in range(num_epochs):
    for batch in train_loader:
        inputs, targets = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()
        outputs = energy_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/1], Loss: nan
